# Getting the most out of NSIDC DAAC data: Harmonizing disparate data for land and sea ice applications

This tutorial will walk you through data discovery, access, and analysis of NSIDC DAAC data...


Here are the steps you will learn in this tutorial:
  
   * Learn the basics of Jupyter Lab and Jupyter Notebook
   * Visualize time and area of interest using NASA Worldview / OpenAltimetry
   * Search for data programmatically by time and area of interest.
   * Determine subsetting and reformatting capabilities for our data of interest.
   * Access and customize data using NSIDC's API service.
       * ICESat-2
       * CETB
       * Ice velocity
       * Passive microwave
   * Plot ATL03 photon data along with higher level ATL06/ATL10 data
   * Filter ICESat-2 data by quality flag
   * Use xarray to extract gridded data along ICESat-2 tracks.
   * Plot data with MODIS true color WMS endpoint. 



### Import Packages

In [ ]:
import requests
import getpass
import socket
import json
import zipfile
import io
import math
import os
import shutil
import pprint
import time
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import fiona
import h5py
import re
import import_ipynb
from ipynb.fs.full.Functions import *
# To read KML files with geopandas, we will need to enable KML support in fiona (disabled by default)
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'
from shapely.geometry import Polygon, mapping
from shapely.geometry.polygon import orient
from statistics import mean
from requests.auth import HTTPBasicAuth

### Introduction to NSIDC and NSIDC data resources



### Search for data programmatically by time and area of interest.


In [ ]:
# Input temporal range 

# Input start date in yyyy-MM-dd format
start_date = '2019-02-22'
# Input start time in HH:mm:ss format
start_time = '00:00:00'
# Input end date in yyyy-MM-dd format
end_date = '2019-05-22'
# Input end time in HH:mm:ss format
end_time = '23:59:59'

temporal = start_date + 'T' + start_time + 'Z' + ',' + end_date + 'T' + end_time + 'Z'
print(temporal)


In [ ]:
# Bounding Box spatial parameter in 'W,S,E,N' format

# Input lower left longitude in decimal degrees
LL_lon = '-64'
# Input lower left latitude in decimal degrees
LL_lat = '66'
# Input upper right longitude in decimal degrees
UR_lon = '-55'
# Input upper right latitude in decimal degrees
UR_lat = '72'

bounding_box = LL_lon + ',' + LL_lat + ',' + UR_lon + ',' + UR_lat
print(bounding_box)

In [ ]:
data_search()

Note that subsetting, reformatting, or reprojecting can alter the size of the granules if those services are applied to your request.

### Input Earthdata Login credentials

An Earthdata Login account is required to access data from the NSIDC DAAC. If you do not already have an Earthdata Login account, visit http://urs.earthdata.nasa.gov to register.

In [ ]:
# Earthdata Login credentials

uid = input('Earthdata Login user name: ')
pswd = getpass.getpass('Earthdata Login password: ')
email = input('Email address associated with Earthdata Login account: ')

### Select the subsetting, reformatting, and reprojection services enabled for your data set of interest.

The NSIDC DAAC supports customization services on many of our NASA Earthdata mission collections. Let's discover whether or not our data set has these services available. If services are available, we will also determine the specific service options supported for this data set and select which of these services we want to request.

In [ ]:
def 

# Query service capability URL 

from xml.etree import ElementTree as ET

capability_url = f'https://n5eil02u.ecs.nsidc.org/egi/capabilities/{short_name}.{latest_version}.xml'

# Create session to store cookie and pass credentials to capabilities url

session = requests.session()
s = session.get(capability_url)
response = session.get(s.url,auth=(uid,pswd))

root = ET.fromstring(response.content)

#collect lists with each service option

subagent = [subset_agent.attrib for subset_agent in root.iter('SubsetAgent')]

# variable subsetting
variables = [SubsetVariable.attrib for SubsetVariable in root.iter('SubsetVariable')]  
variables_raw = [variables[i]['value'] for i in range(len(variables))]
variables_join = [''.join(('/',v)) if v.startswith('/') == False else v for v in variables_raw] 
variable_vals = [v.replace(':', '/') for v in variables_join]

# reformatting
formats = [Format.attrib for Format in root.iter('Format')]
format_vals = [formats[i]['value'] for i in range(len(formats))]
format_vals.remove('')

# reformatting options that support reprojection
normalproj = [Projections.attrib for Projections in root.iter('Projections')]
normalproj_vals = []
normalproj_vals.append(normalproj[0]['normalProj'])
format_proj = normalproj_vals[0].split(',')
format_proj.remove('')
format_proj.append('No reformatting')

# reprojection options
projections = [Projection.attrib for Projection in root.iter('Projection')]
proj_vals = []
for i in range(len(projections)):
    if (projections[i]['value']) != 'NO_CHANGE' :
        proj_vals.append(projections[i]['value'])
        
# reformatting options that do not support reprojection
no_proj = [i for i in format_vals if i not in format_proj]

# SMAP-specific reprojection logic

#L1-L2 reprojection/reformatting options
if short_name == 'SPL1CTB' or 'SPL1CTB_E' or 'SPL2SMA' or 'SPL2SMP' or 'SPL2SMP_E' or 'SPL2SMAP': 
    format_proj = ['GeoTIFF', 'NetCDF4-CF', 'HDF-EOS5']
    no_proj = [i for i in format_vals if i not in format_proj]
elif short_name == 'SPL2SMAP_S' or 'SPL3SMA' or 'SPL3SMP' or 'SPL3SMP_E' or 'SPL3SMAP' or 'SPL3FTA' or 'SPL3FTP' or 'SPL3FTP_E' or 'SPL4SMAU' or 'SPL4SMGP' or 'SPL4SMLM' or 'SPL4CMDL': 
    format_proj = ['No reformatting', 'GeoTIFF', 'NetCDF4-CF', 'HDF-EOS5']
    no_proj = [i for i in format_vals if i not in format_proj]
    
#print service information depending on service availability and select service options
    
if len(subagent) < 1 :
    print('No services exist for', short_name, 'version', latest_version)
    agent = 'NO'
    bbox = ''
    time_var = ''
    reformat = ''
    projection = ''
    projection_parameters = ''
    coverage = ''
else:
    agent = ''
    subdict = subagent[0]
    if subdict['spatialSubsetting'] == 'true':
        ss = input('Subsetting by bounding box, based on the area of interest inputted above, is available. Would you like to request this service? (y/n)')
        if ss == 'y': bbox = bounding_box
        else: bbox = ''
    if subdict['temporalSubsetting'] == 'true':
        ts = input('Subsetting by time, based on the temporal range inputted above, is available. Would you like to request this service? (y/n)')
        if ts == 'y': time_var = start_date + 'T' + start_time + ',' + end_date + 'T' + end_time 
        else: time_var = ''
    else: time_var = ''
    if len(format_vals) > 0 :
        print('These reformatting options are available:', format_vals)
        reformat = input('If you would like to reformat, copy and paste the reformatting option you would like (make sure to omit quotes, e.g. GeoTIFF), otherwise leave blank.')
        # select reprojection options based on reformatting selection
        if reformat in format_proj and len(proj_vals) > 0 : 
            print('These reprojection options are available with your requested format:', proj_vals)
            projection = input('If you would like to reproject, copy and paste the reprojection option you would like (make sure to omit quotes, e.g. GEOGRAPHIC), otherwise leave blank.')
            # Enter required parameters for UTM North and South
            if projection == 'UTM NORTHERN HEMISPHERE' or projection == 'UTM SOUTHERN HEMISPHERE': 
                NZone = input('Please enter a UTM zone (1 to 60 for Northern Hemisphere; -60 to -1 for Southern Hemisphere):')
                projection_parameters = str('NZone:' + NZone)
            else: projection_parameters = ''
        else: 
            print('No reprojection options are supported with your requested format')
            projection = ''
            projection_parameters = ''
    else: 
        reformat = ''
        projection = ''
        projection_parameters = ''

### Access and customize data using NSIDC's API



Intro to API

### Data filtering by quality flag



### Xarray to extract gridded data along ICESat-2 tracks


### Plot data with MODIS true color WMS endpoint

In [ ]:
url = 'https://gibs.earthdata.nasa.gov/twms/epsg4326/best/twms.cgi?request=GetMap&layers=MODIS_Terra_CorrectedReflectance_TrueColor&srs=EPSG:4326&format=image/jpeg&styles=&time=2012-07-09&width=512&height=512&bbox=-18,27,-13.5,31.5'